In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
cd ..

/home/suriya/dcu/Group-Project


# Experiment Tracking and Visualization

In [3]:
import numpy as np
import pandas as pd

In [4]:
from src.features.rssi_distance import extract_feature2, postproc_feature_dicts
from src.featutils import aggregate_features_from_folder

In [5]:
train_dir = "data/tc4tl_training_data_v1/tc4tl/data/train/"
train_key = pd.read_csv("data/tc4tl_training_data_v1/tc4tl/docs/tc4tl_train_key.tsv", sep="\t")
test_dir = "data/tc4tl_data_v5/tc4tl/data/test/"
test_mdata = pd.read_csv("data/tc4tl_data_v5/tc4tl/docs/tc4tl_test_metadata.tsv", sep="\t")
test_key = pd.read_csv("data/tc4tl_test_key/tc4tl/docs/tc4tl_test_key.tsv", sep="\t")

In [6]:
trainset, _ = aggregate_features_from_folder(train_dir, train_key, feat_fn=extract_feature2, postproc_fn=postproc_feature_dicts, verbose=True)
testset, _  = aggregate_features_from_folder(test_dir, test_key, feat_fn=extract_feature2, postproc_fn=postproc_feature_dicts, verbose=True)

100%|██████████| 8423/8423 [00:13<00:00, 620.51it/s]


In [7]:
cg_mapping = {'Y' : 0., 'N' : 1.}
trainset.CoarseGrain = trainset.CoarseGrain.replace(cg_mapping)
testset.CoarseGrain = testset.CoarseGrain.replace(cg_mapping)

In [8]:
features = ['PredictedDistance', 'CoarseGrain']
target = 'Distance'

X_train, y_train = trainset[features].values, trainset[target].values
X_test, y_test = testset[features].values, testset[target].values
labels = trainset[target].unique()

In [9]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier()
rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)
y_prob = rf.predict_proba(X_test)
importances = rf.feature_importances_
indices = np.argsort(importances)[::-1]

In [10]:
importances

array([0.90945141, 0.09054859])

In [11]:
rf.feature_importances_

array([0.90945141, 0.09054859])

## Init wandb

In [12]:
import wandb

In [13]:
run = wandb.init(project="sklearn-integration")

wandb: Currently logged in as: suriyadeepan (use `wandb login --relogin` to force relogin)


## Classifier Plots

In [14]:
wandb.sklearn.plot_roc(y_test, y_prob, labels)

In [15]:
wandb.sklearn.plot_precision_recall(y_test, y_prob, labels)

In [16]:
wandb.sklearn.plot_feature_importances(rf, feature_names=features)

In [17]:
wandb.sklearn.plot_class_proportions(y_train, y_test, labels)

In [19]:
wandb.sklearn.plot_confusion_matrix(y_test, y_pred, labels)

In [21]:
wandb.sklearn.plot_learning_curve(rf, X_train, y_train)

## Weights & Biases Visualization

[suriyadeepan/workspace](https://wandb.ai/suriyadeepan/sklearn-integration/runs/1v15n1em?workspace=user-suriyadeepan)

# Dataset Versioning

In [9]:
import wandb

run = wandb.init(job_type="dataset-creation")
artifact = wandb.Artifact('tc4tl-dataset', type='dataset')

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: suriyadeepan (use `wandb login --relogin` to force relogin)


In [10]:
trainset.to_csv('data/processed/basic.train.csv', index=False)
testset.to_csv('data/processed/basic.test.csv', index=False)

In [11]:
artifact.add_file("data/processed/basic.train.csv")
artifact.add_file("data/processed/basic.test.csv")
run.log_artifact(artifact)

In [12]:
run = wandb.init(job_type="model-training")
artifact = run.use_artifact('tc4tl-dataset:latest')
artifact_dir = artifact.download()

In [18]:
!tree artifacts/

artifacts/
└── tc4tl-dataset:v0
    ├── basic.test.csv
    └── basic.train.csv

1 directory, 2 files


## Log a new version

### Make a new version

In [23]:
from sklearn.preprocessing import MinMaxScaler

scalar = MinMaxScaler()
norm_predicted_distance = scalar.fit_transform(trainset.PredictedDistance.values.reshape(-1, 1))
trainset['NormPredictedDistance'] = norm_predicted_distance

In [24]:
trainset.to_csv("data/processed/basic.train.csv", index=False)

### Save newer version

In [25]:
run = wandb.init(job_type="dataset-creation")
artifact = wandb.Artifact('tc4tl-dataset', type='dataset')
artifact.add_file("data/processed/basic.train.csv")
run.log_artifact(artifact)

### Get newer version of the dataset

In [26]:
run = wandb.init(job_type="model-training")
artifact = run.use_artifact('tc4tl-dataset:latest')
artifact_dir = artifact.download()

In [27]:
!tree artifacts/

artifacts/
├── tc4tl-dataset:v0
│   ├── basic.test.csv
│   └── basic.train.csv
└── tc4tl-dataset:v1
    └── basic.train.csv

2 directories, 3 files
